<a href="https://colab.research.google.com/github/JacobeCode/ML-AI/blob/main/trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mlflow --quiet
!pip install pyngrok --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.6/147.6 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 3.4 MB/s eta 0:00:00


In [7]:
import scipy
import mlflow
import scipy.stats
import pandas
import os
import pickle

import numpy as np

from google.colab import drive
from collections import Counter
from mlflow.models import infer_signature
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, recall_score, f1_score, make_scorer, confusion_matrix, log_loss, precision_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate, StratifiedKFold
from pathlib import Path
from sklearn.model_selection import train_test_split

drive.mount('/content/drive')

database_dir='/MyDrive/Sentiment_Tests/database'

# This is set to save files, data and artifacts to Google Drive without tuneling (download files and visualize/analize localy)
# IMPORTANT - if you want to see artifacts and other data there is need to change "meta.yml" directories to your local equivalents
# Maybe later I'll do automatic version
mlflow.set_tracking_uri("/content/drive/MyDrive/mlruns")

mlflow.set_experiment("RandomForestClassificator")

# Labeling and sorting through data
f=open('/content/drive/MyDrive/Sentiment_Tests/dataset/train.txt', "r")
train_data_unclean=f.read()
train_data=train_data_unclean.split("\n")
for num, item in enumerate(train_data):
    train_data[num]=item.split(";")

data=[]
labels=[]
for item in train_data:
    data.append(item[0])
    labels.append(item[1])

# Labeling and sorting through data (test_data)
f=open('/content/drive/MyDrive/Sentiment_Tests/dataset/test.txt', "r")
test_data_unclean=f.read()
test_data=test_data_unclean.split("\n")
for num, item in enumerate(test_data):
    test_data[num]=item.split(";")

test_dataset=[]
test_labels=[]
for item in test_data:
    test_dataset.append(item[0])
    test_labels.append(item[1])


# Converting simple arrays to DataFrame
train_database=pandas.DataFrame(data=data)
train_database["sentiment"]=labels
train_database.columns=["text_data", "sentiment"]

with mlflow.start_run():
    # Manually setting train_test sets (not with sklearn train_test_split because train/test sets are prepared in dataset)
    X_train=data
    y_train=labels
    X_test=test_dataset
    y_test=test_labels

    params={
        "criterion": "gini",
        "n_estimators": 150,
    }

    # Model creation
    model=RandomForestClassifier(**params)

    # Vectorizing of text data
    count_vec=CountVectorizer()
    X_train_counts=count_vec.fit_transform(X_train)
    X_test_counts=count_vec.transform(X_test)

    # Train fit and prediction of model
    model.fit(X_train_counts, y_train)
    preds_test=model.predict(X_test_counts)

    # Accuracy score
    accuracy=accuracy_score(y_test, preds_test)
    print(accuracy)

    mlflow.log_params(params)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.set_tag("Training Info", "Is it working through Cloud")
    signature=infer_signature(X_train, preds_test)
    model_info=mlflow.sklearn.log_model(
        sk_model=RandomForestClassifier,
        artifact_path="sentiment_model",
        signature=signature,
        input_example=X_train_counts,
        registered_model_name="gini",
    )

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Exception: Run with UUID 03c026f8025545ffb041910e333c8bbb is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True

In [ ]:
# This is addition of tuneling through ngrok to look into training data with use of mlflow without Drive
from pyngrok import ngrok
from google.colab import userdata

get_ipython().system_raw('mlflow ui --port 5000 &')

ngrok.kill()

ngrok.set_auth_token(userdata.get('NGROK_AUTH_TOKEN'))

ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

In [6]:
# Killing existing ngrok processes
from pyngrok import ngrok

ngrok.kill()